<h1> APS 2: Redes Sociais </h1>

Professor: Marcelo Hashimoto

Alunos: Henrique Martinelli Frezzatti, Lívia Sayuri Makuta e Rodrigo Coelho.

<h2> Introdução </h2>

Na presente atividade, a partir de um grafo que foi construído na APS anterior, iremos praticar o básico sobre cálculo de métricas locais de vértices e interpretar a rede e essas métricas. Assim, para lembrar, o dataset que foi escolhido pelo grupo: `soc-pages-sport`, foi retirado do tópico *social networks* do site: https://networkrepository.com/. Essa base de dados escolhida utiliza informações coletadas de páginas do *Facebook* do período de novembro de 2017 e busca mostrar as curtidas mútuas entre essas páginas. Além disso, essas páginas são verificadas e estão concentradas na categoria de esportes, embora os dados não incluam apenas essa categoria. 

Sendo assim, os vértices representam as páginas verificadas do *Facebook* que estão concentradas na categoria esportes, e as arestas são as curtidas mútuas entre elas. Agora, antes de prosseguir para o cálculo das métricas locais, vamos carregar o grafo 
que estará sob a ação de um algoritmo baseado na ideia de *force-directed graph drawing* que utiliza as arestas como uma força gravitacional que aproxima vértices conectados e afasta os desconectados. 

E, para melhorar a visualização utilizaremos a ideia de componente, que é um conjunto máximo de vértices que permite que para quaisquer vértices do conjunto exista um caminho as conectando, isto é, podemos andar de qualquer vértice até outro. Assim, será extraída apenas a maior componente, e isso será feito através da função `gt_clean`.


In [1]:
import netpixi

from netpixi.integration.gt import *
from regression.integration.gt import *

/usr/local/lib/python3.10/site-packages/seaborn/rcmod.py:4: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()


Observação: para rodar a célula abaixo basta tirar os `#` do início das linhas. Deixamos esta célula comentada porque o notebook fica extremamente lento e de difícil manuseio depois de rodá-la, e isso se deve ao fato do grafo ter muitas vértices e arestas. Caso não queira rodar a célula, a visualização pode ser na imagem: `grafoCompletoSemZoom.png` e também com um pouco de zoom na imagem:  `grafoCompletoZoom.png`.




In [2]:
g_completo = gt_load('./versao1.net.gz')
g_completo = gt_clean(g_completo)
m_completo = gt_draw.sfdp_layout(g_completo)
gt_move(g_completo, m_completo)
gt_save(g_completo, 'force-directed.net.gz')
#r = netpixi.render('force-directed.net.gz', infinite=True);

In [3]:
# mudar a cor e o tamanho dos vértices
#r.vertex_default(color=0xFF0000, size=0.7)

# mudar a cor, a espessura e a curvatura das arestas
#r.edge_default(color=0x20B2AA, width=0.08, curve1=1, curve2=1)

#r.vertex_set_key('nome')

![title](./grafoCompletoSemZoom.png)
Imagem do grafo completo com o algoritmo. 
![title](./grafoCompletoZoom.png)
Imagem do grafo com o algoritmo e zoom.

As bolinhas (que nesse grafo acima estão bem pequenas) representam as páginas do *Facebook* e as linhas entre elas representam as arestas. Como pode ser visto, alguns grupos foram formados e isso está explícito em algumas regiões onde há maior concentração de arestas e também de vértices. Para entender melhor o que representam esses grupos, vamos utilizar uma versão reduzida do grafo que também foi montada na APS anterior e que conta com cerca de 2000 arestas e 1873 vértices.

In [4]:
g_reduzido = gt_load('./grafoMenorGrupos.net.gz')
g_reduzido = gt_clean(g_reduzido)
m_reduzido = gt_draw.sfdp_layout(g_reduzido)
gt_move(g_reduzido, m_reduzido)
gt_save(g_reduzido, 'force-directed-grafoMenor.net.gz')
#r = netpixi.render('force-directed-grafoMenor.net.gz');


In [5]:
# mudar a cor e o tamanho dos vértices
r.vertex_default(color=0xFF0000, size=1)

# mudar a cor, a espessura e a curvatura das arestas
r.edge_default(color=0x20B2AA, width=0.1, curve1=1, curve2=1)

r.vertex_set_key('nome')

NameError: name 'r' is not defined

Agora sim podemos analisar melhor o que esses grupos representam e concluir que são páginas de esporte de categorias ou nichos similares que trocam curtidas entre si. Um exemplo é um grupo que é formado pelas páginas: *Minnesota Vikings*, *New England Patriots*, *Los Angeles Chargers*, *Brian Robison*, *Kyle Rudolph*, entre outros. Todos esses são times e jogadores da NFL que é a liga de futebol americano. 

Daqui para frente no decorrer da atividade, utilizaremos apenas o grafo reduzido para poder manuseá-lo e ter uma visualização melhor, entretanto para visualizar o cálculo das métricas  nas tabelas iremos utilizar a base de dados completa para formular um conceito para a métrica de maneira mais geral, e não apenas através desse recorte que fizemos para ajudar na visualização. 

<h2> Cálculo das métricas</h2>

<h3> [a] Cálculo da centralidade degree </h3>

Como visto na APS anterior, cada vértice (ou nó) possui arestas que entram (*in-degree*) e arestas que saem (*out-degree*), e a soma do número de arestas que entram com o número das arestas que saem de um nó é chamado de *degree*. No caso de um grafo não-dirigido, o sentido dessa conexão não importa e podemos simplesmente chamar de *degree*, como é o caso que está sendo trabalhado, e isso porque as conexões entre as páginas do Facebook representam curtidas mútuas entre elas. 

E é por meio do *degree* que podemos calcular a métrica de centralidade *degree* que nada mais é do que o *degree*  de um vértice dividido pelo máximo *degree* possível. No nosso caso podemos pensar no seguinte exemplo: será que o *Minnesota Vikings* tem muitas curtidas mesmo? Qual a relação entre o número de suas curtidas mútuas com à página que tem mais curtidas mútuas? Isso pode nos ajudar a entender a importância da página no contexto que está inserida.



Primeiro vamos carregar os grafos que serão utilizados:

In [51]:
g_reduzido = gt_load('force-directed-grafoMenor.net.gz')
g_completo = gt_load('force-directed.net.gz')

Agora vamos calcular a centralidade *degree*: 

In [52]:
dc_reduzido = gt_total_degree(g_reduzido)
dc_completo = gt_total_degree(g_completo)

#for value in dc:
#    print(value) [debug]

Nos primeiros tutoriais da biblioteca graph-tool, vimos que era possível adicionar propriedade aos vértices de um grafo através do método `add_vp`. E além de adicionar uma propriedade, também é possível adicionar um segundo parâmetro. Logo se passarmos a centralidade como segundo parâmetro, ela será uma propriedade dos vértices do grafo.


In [55]:
g_reduzido.add_vp('degree', dc_reduzido)

g_completo.add_vp('degree', dc_completo)


Abaixo podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira decrescente (veremos a tabela do grafo completo):

In [60]:
gt_data(g_completo).sort_values("degree", ascending = False).head(10)

,nome,degree
id,,
6221,NBA,0.033759
5328,UFC,0.028998
3108,Cristiano Ronaldo,0.022217
11976,NFL,0.021063
11134,LeBron James,0.019332
3334,UEFA Champions League,0.018755
11881,John Doomsday Howard MMA,0.018539
8137,Leo Messi,0.016879
13571,Martyn Ashton,0.016230


Abaixo também podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira crescente (veremos a tabela do grafo completo):

In [57]:
gt_data(g_completo).sort_values("degree", ascending = True).head(10)

,nome,degree
id,,
538,Beşiktaş English,0.000072
9556,한화이글스,0.000072
6128,Brandão,0.000072
12193,ekaterina gamova,0.000072
11643,"""Juan Manuel """"Cochito"""" Lopez""",0.000072
5014,Alessio Romagnoli,0.000072
7226,GIANTS Netball,0.000072
947,Emmanuel Bonaventure Dennis,0.000072
6873,Velothon,0.000072


Por fim, podemos ver essa propriedade visualmente no grafo e através do método `vertex_scale` é possível utilizar o `degree` como tamanho.

In [ ]:
#m = gt_draw.sfdp_layout(g_reduzido)
#gt_move(g_reduzido, m)
#gt_save(g_reduzido, 'centralidadeDegree.net.gz')
#r = netpixi.render('centralidadeDegree.net.gz')

In [ ]:
r.vertex_set_key('nome')
r.vertex_scale('degree', 1, 20)

A centralidade *degree* no presente contexto poderia ser definida como a popularidade da página do *Facebook* que pode ser tanto uma pessoa quanto uma organização no nicho em que está inserida. Além disso, podemos analisar também que dado o fato das arestas serem curtidas mútuas, elementos com grande centralidade *degree* são membros ativos na comunidade que estão inseridos - no sentido de curtir outras páginas. Por exemplo, Antoine Walker é um ex-jogador de basquete e podemos ver que suas conexões estão mais relacionadas a outros jogadores e também páginas de time de basquete, assim, por essa métrica podemos dizer que ele seria uma pessoa popular na comunidade de basquete e que provavelmente também é membro ativo dessa comunidade na rede social curtindo outras páginas com interesses semelhantes ou até de outros nichos.

Aparentemente os maiores vértices estão em posições interessantes. Isso porque a maior parte deles parece ser a posição central do *cluster* (agrupamento) em que estão inseridos e também na rede central - dependendo do quão maior é sua centralidade *degree*, como se fossem regiões de influência naquele nicho e/ou até mesmo na rede de esportes.


<h3> [b] Cálculo da centralidade betweeness </h3>

A centralidade betweeness, por sua vez, diz respeito a quantas vezes um vértice acaba sendo intermediário entre dois outros. Parece confuso, mas basta pensar que seria uma sequência de vértices e passos para sair de A e ir até B andando pelas arestas. Entretanto, é importante lembrar que não é qualquer deslocamento, nesse deslocamente não se pode passar pela mesma aresta mais de um vez e nem passar pelo mesmo vértice mais de uma vez. Além disso, esse deslocamento precisa ter o menor comprimento - isto é, o número de arestas pelas quais um caminho passa deve ser o menor possível.

Basta pensar que para uma página de futebol curtir uma de esqui talvez tenham algumas outras páginas intermediárias que acabaram sendo uma "ponte" para que essa curtida mútua acontecesse. E é isso que o *betweeness* representa, adicionando as condições citadas anteriormente.



Primeiro vamos carregar os grafos que serão utilizados:

In [66]:
g_reduzido = gt_load('force-directed-grafoMenor.net.gz')

g_completo = gt_load('force-directed.net.gz')

Agora vamos calcular a centralidade *betweenness*:

In [67]:
from graph_tool import centrality

bc_reduzido, _ = centrality.betweenness(g_reduzido)

bc_completo, _ = centrality.betweenness(g_completo)
#for value in bc:
#    print(value) [debug]

Depois podemos adicionar essa propriedade aos vértices do grafo através do método `add_vp`

In [68]:
g_reduzido.add_vp('betweenness', bc_reduzido)

g_completo.add_vp('betweenness', bc_completo)


Abaixo podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira decrescente (veremos a tabela do grafo completo):

In [69]:
gt_data(g_completo).sort_values("betweenness", ascending = False).head(10)

,nome,betweenness
id,,
6221,NBA,0.056742
5328,UFC,0.038914
3108,Cristiano Ronaldo,0.033980
11976,NFL,0.029374
1615,Paralympic Games,0.025201
4938,Monster Energy,0.022855
12449,Team USA,0.022840
3334,UEFA Champions League,0.021986
6929,Vernon Davis,0.021089


Abaixo também podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira crescente (veremos a tabela do grafo completo):

In [70]:
gt_data(g_completo).sort_values("betweenness", ascending = True).head(10)

,nome,betweenness
id,,
13792,Petrix Barbosa,0.0
12684,Martin Schmitt,0.0
848,Jacketsonline.com,0.0
2183,Tai Aguero,0.0
5817,Michael Annett,0.0
1894,Willi Wildpark,0.0
12302,Avanti Palmeiras,0.0
3560,KamiKat,0.0
9856,Leones del Caracas,0.0


In [14]:
#m = gt_draw.sfdp_layout(g_reduzido)
#gt_move(g_reduzido, m)
#gt_save(g_reduzido, 'centralidadeBetweenness.net.gz')
#r = netpixi.render('centralidadeBetweenness.net.gz')

In [13]:
r.vertex_set_key('nome')
r.vertex_scale('betweenness', 1, 20)

Em relação ao significado da centralidade *betweenness*, tem-se que essa é uma página em comum que conecta diferentes comunidades, e isso pode acontecer devido a informações compartilhadas, como lugares em comum, estar no mesmo grupo do *Facebook*, ser marcado em uma publicação naquela página, entre outros. E essa informação pode ser utilizada na ferramenta  "Páginas que você talvez curta" ou "Páginas que outras pessoas também curtiram". 

Por exemplo, olhando o grafo anterior, Charlotte Hornets é um time de basquete americano que embora não fosse um dos mais populares e nem ativos na iteração por curtidas (ou seja, não possui uma centralidade *degree* alta), acabou por ter um *betweenness* alto, e isso porque essa página compartilha curtidas mútuas com duas páginas muito populares: NASCAR e Antoine Walker, assim, essa página acaba por conectar essas duas páginas de grupos distintos em um caminho mais curto. Ela não será necessariamente recomendada para o usuário curtir, mas ela é uma intermediária que conecta diferentes comunidades e que possivelmente pode ser usada por algoritmos do *Facebook* para traçar caminhos e sugerir que alguém que curtiu NASCAR possa querer curtir Antoine Walker. Inclusive essa relação entre NASCAR e Antoine Walker com Hornets pode ter acontecido pelo fato de que um dos circuitos mais importantes de NASCAR seja na cidade de Charlotte, que também é a cidade do time de basquete Charlotte Hornets.

Logo, quanto mais uma página conecta comunidades de esporte no *Facebook* maior é seu *betweenness* e quanto mais parte de um grupo ela é, sem se conectar com outros, menor é seu *betweenness*. 

Agora em relação à posição que esses vértices maiores estão, podemos dizer que são posições extremamente interessantes. Isso porque, reiterando, essas páginas são as responsáveis por conectar páginas de grupos e interesses distintos. Como citado no exemplo anterior, temos Charlotte Hornets conectando NASCAR (associação automobilística) com Antoine Walker (ex-jogador de basquete).



<h3> [c] Cálculo da centralidade closeness </h3>

O closeness por sua vez é o inverso da soma de distâncias de um vértice aos outros, ou seja, tenta medir o quão próximo um vértice está do outro levando em consideração o menor comprimento possível de um caminho de um ponto até outro. Assim, quanto mais central um nó é, mais próximo será de todos os outros nós. No contexto do *Facebook*, podemos pensar que se o *closeness* de uma página for alto, é porque ela é bem próxima de muitas outras, ou seja o número de conexões ou passos para sair dela e ir para outra é baixo - indicando proximidade.

Primeiro vamos carregar os grafos que serão utilizados:

In [99]:
g_reduzido = gt_load('force-directed-grafoMenor.net.gz')

g_completo = gt_load('force-directed.net.gz')

Agora vamos calcular a centralidade *closeness*:

In [100]:
from graph_tool import centrality

cc_reduzido = centrality.closeness(g_reduzido)

cc_completo = centrality.closeness(g_completo)
#for value in cc:
#    print(value) [debug]

Depois podemos adicionar essa propriedade aos vértices do grafo através do método `add_vp`:

In [101]:
g_reduzido.add_vp('closeness', cc_reduzido)

g_completo.add_vp('closeness', cc_completo)

Abaixo podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira decrescente (veremos a tabela do grafo completo):

In [102]:
gt_data(g_completo).sort_values("closeness", ascending = False).head(10)

,nome,closeness
id,,
6221,NBA,0.349670
3108,Cristiano Ronaldo,0.344542
9126,David Beckham,0.343569
11134,LeBron James,0.343288
6929,Vernon Davis,0.342864
4065,Kobe Bryant,0.341908
10583,Ivy Martin,0.339979
8137,Leo Messi,0.330071
774,"""Coby Miller",0.330040


Abaixo também podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira crescente (veremos a tabela do grafo completo):


In [103]:
gt_data(g_completo).sort_values("closeness", ascending = True).head(10)

,nome,closeness
id,,
872,"""U.S. Army Corps of Engineers",0.123673
1079,Luis Scola,0.137355
7246,Finn 'karrigan' Andersen,0.137355
289,Yohei Uchino,0.137355
6688,Denis 'denis' Howell,0.137355
7353,Ohio University Men's Basketball,0.139803
5014,Alessio Romagnoli,0.141119
8371,Alysia Rissling,0.141121
7176,Christian Mathenia,0.141122


In [108]:
#m = gt_draw.sfdp_layout(g_reduzido)
#gt_move(g_reduzido, m)
#gt_save(g_reduzido, 'centralidadeCloseness.net.gz')
#r = netpixi.render('centralidadeCloseness.net.gz')

In [107]:
r.vertex_set_key('nome')
r.vertex_scale('closeness', 0.5, 3)

Em relação ao *closeness* não conseguimos enxergar uma interpretação conceitual plausível, isso porque todos os vértices possuem *closeness* parecidos e de maneira geral estão próximos entre si em seus grupos e também fora deles, isto é, nessa rede de páginas no *Facebook* não há muito como as páginas de esporte terem distâncias muito grandes entre si, pois embora existam diferentes categorias de esporte, muitas delas estão conectadas de alguma forma por alguma similaridade, então não há um conceito que defina bem essa métrica e que faça sentido para comparações. 

Apenas algumas páginas tem um *closeness* um pouco maior do que as outras mas é porque de maneira geral são centrais na rede toda, ou seja, são páginas que provavelmente são as mais famosas no mundo dos esportes e até fora dele também, dessa forma, acabam tendo maior proximidade com as outras vértices por conta da sua posição central, que é interessante.


<h3>[d] Clustering coefficient</h3>

Agora saindo um pouco das métricas de centralidade, também é interessante analisar o quão agrupado as redes estão, ou seja, o quanto um vértice parece fazer parte de um grupo. Na análise visual fizemos isso com um algoritmo baseado na ideia de *force-directed graph*, mas saindo desse campo podemos verificar isso também através de uma métrica.

Assim, o *clustering coefficient* vai analisar o quanto uma página de *Facebook* compartilha de curtidas mútuas com todas as páginas mais próximas de si (do seu "grupinho"), isso vai ser interessante para descobrir o quão conectadas essas páginas de interesses parecidos estão entre si.


Primeiro vamos carregar os grafos que serão utilizados:

In [109]:
g_reduzido = gt_load('force-directed-grafoMenor.net.gz')

g_completo = gt_load('force-directed.net.gz')

Agora vamos calcular o *clustering coefficient*:

In [110]:
from graph_tool import clustering

lc_reduzido = clustering.local_clustering(g_reduzido)

lc_completo = clustering.local_clustering(g_completo)

#for value in lc:
#    print(value) [debug]

Depois podemos adicionar essa propriedade aos vértices do grafo através do método `add_vp`:

In [111]:
g_reduzido.add_vp('clustering', lc_reduzido)

g_completo.add_vp('clustering', lc_completo)

Abaixo podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira decrescente (veremos a tabela do grafo completo):

In [112]:
gt_data(g_completo).sort_values("clustering", ascending = False).head(10)

,nome,clustering
id,,
13792,Petrix Barbosa,1.0
1240,Andreas Rojewski,1.0
1978,Midol Bastien,1.0
8484,FC Barcelona Futbol Sala,1.0
7664,Nia Künzer,1.0
11472,Rey Maualuga,1.0
137,Julian Reister,1.0
2611,許基宏 粉絲團,1.0
13437,Xavier Woods,1.0


Abaixo também podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira crescente (veremos a tabela do grafo completo):

In [113]:
gt_data(g_completo).sort_values("clustering", ascending = True).head(10)

,nome,clustering
id,,
1947,"""Quinn """"Moose"""" Ojinnaka""",0.0
1372,Ogden Raptors,0.0
9385,Sarah Reinertsen,0.0
3889,Denzel Valentine,0.0
9427,Ferencvárosi Torna Club,0.0
7186,Fernando Casanova,0.0
8243,Nabil Bahoui,0.0
4946,Ashley Nee,0.0
5119,FC Basel 1893 Esports,0.0


In [116]:
#m = gt_draw.sfdp_layout(g_reduzido)
#gt_move(g_reduzido, m)
#gt_save(g_reduzido, 'clustering.net.gz')
#r = netpixi.render('clustering.net.gz')

In [115]:
r.vertex_set_key('nome')

r.vertex_scale('clustering', 1, 10)

No grafo que utiliza arestas e vértices reduzidas são poucos os casos que o *clustering* é maior do que 0, indicando que os membros daquele grupo estão interligados. Mas se rodarmos esse cálculo do coeficiente para a base de dados inteira, percebemos que muitos grupos estão interligados, e esses grupos que são interligados possuem interesses em esportes parecidos. Por exemplo, temos um *cluster* com Mario Gotze, Xabi Alonso, Franck Ribéry, entre outros jogadores que mutuamente se curtem pois jogavam todos no mesmo time: o Bayern de Munique.

Assim, o conceito de *clustering* pode significar pessoas que fazem parte de um mesmo grupo, seja de um mesmo time, ou que jogaram ou gostam de um mesmo esporte. Além disso, a posição que eles se encontram é interessante, isso por estarem representando um grupo de interesses em comum ou até mesmo um time, como no caso do grupo formado pelos jogadores do Bayern. E no geral sua posição tende a ser concentrada, o que de fato indica a formação de "panelinhas", como também foi visto na APS anterior na análise visual. 

<h3>[e] Lacunas estruturais </h3>

Por fim as *lacunas estruturais* estão associadas a uma teoria que vê uma aresta como um possível investimento, ou seja, se tenho conexão com algum vértice, quero que essa conexão me traga recursos diferentes do que já tenho, para evitar a redundância. Assim, tenho redundância se, por exemplo, `u` está conectado com `w` e `v` está conectado com `w`, já que `u` pode obter informações e recursos de `v` através de `w` e também em relação ao `w`, já que `u` pode obter recursos e informações de `w` através de `v`. Esses dois exemplos representam dos tipos de redundância para grafos dirigidos, mas para grafos não-dirigidos elas são essencialmente a mesma coisa.

E algumas formas de medir isso são através do tamanho efetivo de de restrição. A primeira penaliza o *degree* sempre que acontecerem redundâncias e a segunda mostra o quanto os recursos de uma página estão restringidos por suas relações (*diminui conforme o investimento em uma relação diminui,* mas, ao mesmo tempo, *aumenta conforme essa conexão tem mais redundância*).


Primeiro vamos carregar os grafos que serão utilizados:

In [117]:
g_reduzido = gt_load('force-directed-grafoMenor.net.gz')

g_completo = gt_load('force-directed.net.gz')

Agora vamos calcular o *effective size*:

In [118]:
es_reduzido = gt_effective_size(g_reduzido)

es_completo = gt_effective_size(g_completo)

#for value in es:
#    print(value) [debug]

Depois podemos adicionar essa propriedade aos vértices do grafo através do método `add_vp`:

In [119]:
g_reduzido.add_vp('effsize', es_reduzido)

g_completo.add_vp('effsize', es_completo)

Abaixo podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira decrescente (veremos a tabela do grafo completo):

In [120]:
gt_data(g_completo).sort_values("effsize", ascending = False).head(10)

,nome,effsize
id,,
6221,NBA,454.662393
5328,UFC,392.263682
3108,Cristiano Ronaldo,303.649351
11976,NFL,285.246575
11134,LeBron James,261.126866
3334,UEFA Champions League,251.292308
11881,John Doomsday Howard MMA,249.210117
8137,Leo Messi,229.837607
13571,Martyn Ashton,219.204444


Abaixo podemos ver as propriedades dos vértices formatadas como um dataframe do pandas de maneira crescente (veremos a tabela do grafo completo):

In [121]:
gt_data(g_completo).sort_values("effsize", ascending = True).head(10)

,nome,effsize
id,,
13792,Petrix Barbosa,1.0
8964,Kai Otton 'Ottz',1.0
5795,Justin Morneau,1.0
8183,AAU Basketball,1.0
13439,Willy Santos,1.0
6628,Will Bayley,1.0
1911,Eric Bailly,1.0
3832,Cyc Fitness,1.0
12938,Dávid Vojvoda,1.0


In [124]:
#m = gt_draw.sfdp_layout(g_reduzido)
#gt_move(g_reduzido, m)
#gt_save(g_reduzido, 'effsize.net.gz')
#r = netpixi.render('effsize.net.gz')

In [123]:
r.vertex_set_key('nome')

r.vertex_scale('effsize', 1, 20)

Em relação ao Effective Size, pode-se levar em conta justamente o que é dito no próprio nome da métrica, uma vez que, ao analisarmos a base de dados completa (por meio da tabela com os maiores e menores `effective size`), é possível analisar que páginas do Facebook com Effective size altos representam instituições, organizações ou personalidades muito influentes, não apenas no meio em que atuam, mas em uma escala muito maior e, muitas vezes, em escala global. Por exemplo, o Lebron James ou Cristiano Ronaldo atuam não apenas em suas áreas, como basquete e futebol, respectivamente, mas também em diversas outras, levando a pessoas curtirem suas páginas no Facebook mesmo que não acompanhem o esporte ou a pessoa em si, curtindo meramente pela fama ou importância social. 

Além disso, podemos concluir que nós com Effective Size baixos podem ser personalidades famosas, porém, acabam sendo muito mais nichadas do que as outras, gerando, muitas vezes, diversas conexões, mas a possibilidade das pessoas que curtem essas páginas se conhecerem é muito maior do que quando o *effective size* é alto. 

Assim, o conceito de effective size pode ser a fama e a influência das páginas em si, de modo que quanto maior mais influentes e conhecidas são e que quanto menor mais nichadas são.

Em relação a posição em que se encontram, temos que as maiores vértices são centrais na base de dados completa e também no meio mais geral que se encontram: por exemplo, Cristiano Ronaldo é central na base de dados e também no seu meio que é o futebolístico, e por conta de ter tanta influência e também fora de seu meio, isso também explica o porquê de talvez não apresentarem tantas redundâncias que abaixem de fato seu *effective size*.

**Observação**: O código para gerar o grafo completo não foi utilizado pois demanda muito processamento do computador e acaba deixando o Jupyter muito lento.

Agora vamos repetir tudo que foi feito anteriormento para o *constraint*

In [125]:
g_reduzido = gt_load('force-directed-grafoMenor.net.gz')

g_completo = gt_load('force-directed.net.gz')

In [126]:
ct_reduzido = gt_constraint(g_reduzido)

ct_completo = gt_constraint(g_completo)

#for value in ct:
#    print(value) [debug]

In [127]:
g_reduzido.add_vp('constraint', ct_reduzido)

g_completo.add_vp('constraint', ct_completo)

In [128]:
gt_data(g_completo).sort_values("constraint", ascending = False).head(10)

,nome,constraint
id,,
5823,Sean SS Smith,1.0
7475,Shelley Olds,1.0
6143,Daniel Wagner,1.0
11684,Emma Foy and Laura Thompson - Paracyclists,1.0
4049,Angy Eiter,1.0
6013,Hanley Ramirez,1.0
11249,Hugo Calderano,1.0
10981,Sandra Gerber,1.0
133,Real Estelí FC,1.0


In [129]:
gt_data(g_completo).sort_values("constraint", ascending = True).head(10)

,nome,constraint
id,,
3108,Cristiano Ronaldo,0.004356
11134,LeBron James,0.005508
6221,NBA,0.005713
8137,Leo Messi,0.005766
11976,NFL,0.006064
3334,UEFA Champions League,0.006716
12721,FIFA World Cup,0.007079
12449,Team USA,0.007250
4065,Kobe Bryant,0.007262


In [132]:
#m = gt_draw.sfdp_layout(g_reduzido)
#gt_move(g_reduzido, m)
#gt_save(g_reduzido, 'constraint.net.gz')
#r = netpixi.render('constraint.net.gz')

In [131]:
r.vertex_set_key('nome')

r.vertex_scale('constraint', 0.5, 2)

Em relação ao *constraint*, podemos dizer que indica a mesma coisa que o *effective size*, só que de maneira diferente. Ou seja, quanto maior a restrição, mais a página está sendo restringida por suas conexões. Nesse caso, podemos perceber que páginas de organizações e instituições esportivas bem como personalidades do esporte que são conhecidas mundialmente possuem *constraints* extremamente baixos, as páginas que possuem restrições altas são exatamente aquelas que possuem *effective size* baixo, isto é, aquelas que são páginas mais nichadas na categoria esportiva em que estão.

Assim, as conclusões para o constraint são as mesmas obtidas ao calcular o *effective size*.

Logo, temos que as lacunas estruturais indicam a relevância da página, sua fama e influência no âmbito dos esportes, de tal forma que quanto mais influentes essas páginas são, isto indica que mais globalmente conhecidas elas são superando os seus grupos locais que normalmente estão associados a categoria de esportes que fazem parte. E quanto menos relevantes e menos famosas elas são, mais nichadas são e mais restritas a sua categoria esportiva elas estão.